### google vision api 사용하여 이미지 라벨 추출

- pip install --upgrade google-cloud-vision
- 사용자 인증정보 > 서비스 계정 키 만들기 > json방식으로 생성
- 사용자 계정 키 json파일을 GOOGLE_APPLICATION_CREDENTIALS  환경변수로 등록



In [3]:
from google.cloud import vision
from google.protobuf import json_format
import argparse
import io
import re

In [13]:
def detect_labels_uri(uri):
    """Detects labels in the file located in Google Cloud Storage or on the
    Web."""
    client = vision.ImageAnnotatorClient()
    image = vision.types.Image()
    image.source.image_uri = uri

    response = client.label_detection(image=image)
    labels = response.label_annotations
    print('Labels:')

    for label in labels:
        #print(label.description)
        print(label.description, label.score)
        
def detect_labels(path):
    """Detects labels in the file."""
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.label_detection(image=image)
    labels = response.label_annotations
    print('Labels:')

    for label in labels:
        #print(label)
        print(label.description, label.score)

In [16]:
def detect_web_uri(uri):
    """Detects web annotations in the file located in Google Cloud Storage."""
    client = vision.ImageAnnotatorClient()
    image = vision.types.Image()
    image.source.image_uri = uri

    response = client.web_detection(image=image)
    annotations = response.web_detection

    if annotations.best_guess_labels:
        for label in annotations.best_guess_labels:
            print('\nBest guess label: {}'.format(label.label))

    if annotations.pages_with_matching_images:
        print('\n{} Pages with matching images found:'.format(
            len(annotations.pages_with_matching_images)))

        for page in annotations.pages_with_matching_images:
            print('\n\tPage url   : {}'.format(page.url))

            if page.full_matching_images:
                print('\t{} Full Matches found: '.format(
                       len(page.full_matching_images)))

                for image in page.full_matching_images:
                    print('\t\tImage url  : {}'.format(image.url))

            if page.partial_matching_images:
                print('\t{} Partial Matches found: '.format(
                       len(page.partial_matching_images)))

                for image in page.partial_matching_images:
                    print('\t\tImage url  : {}'.format(image.url))

    if annotations.web_entities:
        print('\n{} Web entities found: '.format(
            len(annotations.web_entities)))

        for entity in annotations.web_entities:
            print('\n\tScore      : {}'.format(entity.score))
            print(u'\tDescription: {}'.format(entity.description))

    if annotations.visually_similar_images:
        print('\n{} visually similar images found:\n'.format(
            len(annotations.visually_similar_images)))

        for image in annotations.visually_similar_images:
            print('\tImage url    : {}'.format(image.url))

In [18]:
from IPython.display import Image
from IPython.core.display import HTML 

url = "http://i.011st.com/ex_t/R/400x400/1/85/0/src/pd/17/8/3/0/1/4/6/OKblz/1409830146_B.jpg"
detect_labels_uri(url)
detect_web_uri(url)
Image(url= url)

Labels:
product 0.7923773527145386
product 0.7406001091003418
ingredient 0.5950561165809631
cream 0.5040691494941711

Best guess label: instant rice korea

10 Pages with matching images found:

	Page url   : http://nj.hmart.com/shop/ready-to-serve/pre-cooked-rice/
	1 Partial Matches found: 
		Image url  : http://nj.hmart.com/media/catalog/product/cache/7/small_image/295x295/0dc2d03fe217f8c83829496872af24a0/l/-/l-10807176151119_3.jpg

	Page url   : https://shopee.sg/-5packs-CJ-Cooked-Rice-and-barley-Rice-210g-305-kcal-instant-rice-korean-i.8337507.701210805
	1 Partial Matches found: 
		Image url  : https://cf.shopee.sg/file/e8b44d4570baecf8d3805819299c0a40

	Page url   : https://www.amazon.com/Annie-Chuns-Cooked-Sticky-Gluten-Free/dp/B000GPTC8U
	1 Partial Matches found: 
		Image url  : https://images-na.ssl-images-amazon.com/images/I/61bO5umaPkL._AC_UL160_SR160,160_.jpg

	Page url   : https://shopee.sg/-5packs-CJ-Cooked-White-Rice-200g-instant-rice-korean-food-300-kcal-fas-i.8337507.701

In [19]:
url = "http://image.auction.co.kr/itemimage/15/c8/18/15c818b6b6.jpg"
detect_labels_uri(url)
detect_web_uri(url)
Image(url= url)

Labels:
product 0.8748633861541748
product 0.7477615475654602
drink 0.537593424320221
commodity 0.5337325930595398

Best guess label: drink

10 Pages with matching images found:

	Page url   : https://www.na100shop.com:433/goodsList.do?c1=287&c2=291
	1 Full Matches found: 
		Image url  : https://www.na100shop.com:433/pds/goods/goods/6ce8418386de458b94ba33dd678bc4af_thum_file1.jpg

	Page url   : https://www.na100shop.com:433/goodsList.do?c1=287
	1 Full Matches found: 
		Image url  : https://www.na100shop.com:433/pds/goods/goods/6ce8418386de458b94ba33dd678bc4af_thum_file1.jpg

	Page url   : https://www.na100shop.com:433/goodsView.do?gidx=5477
	1 Full Matches found: 
		Image url  : https://www.na100shop.com:433/pds/goods/goods/6ce8418386de458b94ba33dd678bc4af_thum_file1.jpg

	Page url   : http://gopen.kr/spb3/sshop3/list.php?page=12
	1 Partial Matches found: 
		Image url  : http://gopen.kr/spb3/sshop3/download.php?uid=9127105&db=gift&upfile=t2&mode=origin&notfound=any

	Page url   : http: